# Gallery Example: M/M/1-PS Feedback Queue

This example demonstrates an M/M/1 queueing system with Processor Sharing and feedback:
- **Arrivals**: Exponential (Poisson process)
- **Service**: Exponential
- **Servers**: 1 server
- **Scheduling**: Processor Sharing (PS)
- **Feedback**: Jobs may return to the queue with probability p

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_mm1_ps_feedback(p=1/3): Network {
    """Create M/M/1-PS-Feedback queueing model"""
    val model = Network("M/M/1-PS-Feedback")
// Block 1: nodes
    val source = Source(model, "Source")
    val queue = Queue(model, "Queue", SchedStrategy.PS)  # Processor Sharing
    val sink = Sink(model, "Sink")
// Block 2: classes
    val oclass1 = OpenClass(model, "Class1")
// Exponential arrivals and service
    source.setArrival(oclass1, Exp.fitMean(1))
    queue.setService(oclass1, Exp.fitMean(0.5))
// Block 3: topology with feedback (probability p)
    val P = model.initRoutingMatrix()
    P.addRoute(oclass1, source, queue, 1.0)
    P.addRoute(oclass1, queue, queue, p)        # p probability to loop back
    P.addRoute(oclass1, queue, sink, 1 - p)     # 1-p probability to exit
    model.link(P)
    
    return model
// Create the model with default p=1/3
val model = gallery_mm1_ps_feedback()

## About M/M/1-PS Feedback

With feedback probability p = 1/3:
- Mean visits per job = 1 / (1 - 1/3) = 1.5
- External arrival rate = 1
- Effective arrival rate at queue = 1 × 1.5 = 1.5
- Service rate = 2
- Utilization = 1.5 / 2 = 0.75

**Processor Sharing benefits**:
- Fair allocation regardless of job service requirements
- Response time scales linearly with job size
- Good for interactive systems with feedback

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// MVA Solver
val solver_mva = MVA(model)
val avg_table_mva = solver_mva.avgTable
println("\nMVA Solver:")
println(avg_table_mva)
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 15)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)